# Plot Figure 4 (a) & (b)

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Load wind stress data
u = xr.open_mfdataset('/mnt/d/Data/u10.nc').sel(longitude=slice(30, 60), latitude=slice(-60, -67)).u10
v = xr.open_mfdataset('/mnt/d/v10.nc').sel(longitude=slice(30, 60), latitude=slice(-60, -67)).v10

# Calculate wind stress components
wd = (u**2 + v**2) ** 0.5
cd = 1.3 * (10**-3)
tau_x = 1.225 * cd * u * abs(u)
tau_y = 1.225 * cd * v * abs(v)

# Calculate grid spacing
dlat = u.latitude.diff('latitude')
dlon = u.longitude.diff('longitude')
dy_m = np.abs(111320 * dlat)
dx_m = 111320 * dlon * np.cos(np.deg2rad(u.latitude))

# Calculate wind stress curl
d_tau_y_dx = tau_y.differentiate('longitude') / dx_m
d_tau_x_dy = tau_x.differentiate('latitude') / dy_m
wind_stress_curl = d_tau_y_dx - d_tau_x_dy

# Calculate monthly climatology for each month across all years
monthly_climatology = wind_stress_curl.groupby("valid_time.month").mean("valid_time")

# Calculate anomalies by subtracting the climatology from each month's data
wind_stress_curl_anomalies = wind_stress_curl.groupby("valid_time.month") - monthly_climatology

# Average the anomalies over latitude and longitude to get a single time series
wind_stress_curl_anomalies_avg = wind_stress_curl_anomalies.mean(dim=('longitude', 'latitude'))


window_size = 90  # Define the window size for smoothing (e.g., 90-day moving average)
curl_anomalies_smooth = wind_stress_curl_anomalies_avg.rolling(valid_time=window_size, center=True).mean()

# Load polynya area data
polynya_df = pd.read_csv('/mnt/d/Data/Polynya Area/AMSR/Polynya_Area_30.csv')
polynya_df['Timestamp'] = pd.to_datetime(polynya_df['Timestamp'])

# Filter data for June to September for each year
polynya_df = polynya_df[(polynya_df['Timestamp'].dt.month.isin([6, 7, 8, 9]))]
polynya_df = polynya_df[(polynya_df['lat'] >= -67) & (polynya_df['lat'] <= -60) &
                        (polynya_df['lon'] >= 30) & (polynya_df['lon'] <= 60)]


# Find the daily maximum polynya extent for each day in the selected months and years
polynya_df['Date'] = polynya_df['Timestamp'].dt.floor('D')  # Group by each day
daily_polynya_max = polynya_df.groupby('Date')['total_area'].sum().reset_index()  # Sum the area per day

# Find the maximum daily polynya area for each year, keeping June to September
daily_polynya_max['Year'] = daily_polynya_max['Date'].dt.year
max_polynya_per_year = daily_polynya_max.loc[daily_polynya_max.groupby('Year')['total_area'].idxmax()]
max_polynya_per_year['total_area'] = max_polynya_per_year['total_area'] / (10**9)  # Convert to square kilometers

# Load SAM index data
sam_df = pd.read_csv('/mnt/d/Data/SAM.txt', delim_whitespace=True, index_col=0)
sam_df.columns = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']

# Reshape the SAM data to have a DateTime index
sam_long = sam_df.stack().reset_index()
sam_long.columns = ['Year', 'Month', 'SAM']
sam_long['Month'] = pd.to_datetime(sam_long['Month'], format='%b').dt.month
sam_long['Date'] = pd.to_datetime(sam_long[['Year', 'Month']].assign(DAY=1))
sam_long = sam_long.set_index('Date').sort_index()


# Plotting
fig, (ax1, ax3) = plt.subplots(2, 1, figsize=(18, 10), sharex=True, gridspec_kw={'height_ratios': [2, 1]})

# Plot smoothed wind stress curl anomalies on primary y-axis
ax1.plot(curl_anomalies_smooth['valid_time'], curl_anomalies_smooth * 10**5, color='tab:blue', linewidth=2, linestyle='-', label='Smoothed Wind Stress Curl Anomaly (10$^{-5}$ N m$^{-3}$)')
ax1.invert_yaxis()

# Plot polynya area as bars on secondary y-axis, based on the month of max extent
ax2 = ax1.twinx()
ax2.bar(max_polynya_per_year['Date'], max_polynya_per_year['total_area'], color='tab:orange', alpha=0.6, width=50, label='Max Winter Polynya Extent (x $10^{3}$ km²)')

# Format the x-axis for month and year
ax1.xaxis.set_major_locator(mdates.YearLocator())  # Set major ticks to years
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))  # Format ticks to display year and month
fig.autofmt_xdate(rotation=45)  # Rotate date labels

# Set x-axis limits to match the data range
ax1.set_xlim(curl_anomalies_smooth['valid_time'].min(), curl_anomalies_smooth['valid_time'].max())

# Add grid, labels, and title
ax1.grid(True, linestyle='--', alpha=0.7)
ax1.set_ylabel('Wind Stress Curl Anomaly (x 10$^{-5}$ N m$^{-3}$)', fontsize=15, color='tab:blue')
ax2.set_ylabel('Polynya Area (x $10^{3}$ km²)', fontsize=15, color='tab:orange')


ax1.tick_params(axis='both', which='major', labelsize=15)
ax2.tick_params(axis='y', labelsize=15)


ax3.plot(sam_long.index, sam_long['SAM'], color='tab:green', linewidth=1.5, linestyle='-', label='SAM Index')
ax3.axhline(0, color='gray', linestyle='--', linewidth=1)  
ax3.set_ylabel('SAM Index', fontsize=15, color='tab:green')
ax3.grid(True, linestyle='--', alpha=0.5)
ax3.set_ylim(-6, 6)

ax3.tick_params(axis='both', which='major', labelsize=15)



ax4 = ax3.twinx()  
ax4.bar(
    max_polynya_per_year['Date'],
    max_polynya_per_year['total_area'],
    color='tab:orange',
    alpha=0.6,
    width=50,
    label='Max Winter Polynya Extent (x $10^{3}$ km²)'
)
ax4.set_ylabel('Polynya Area (x $10^{3}$ km²)', fontsize=15, color='tab:orange')


ax4.tick_params(axis='y', labelsize=15)

plt.tight_layout()


plt.savefig('/home/soumya/Backup/Plots/Cosmonaut Polynya/curl_anomaly_max_polynya_extent_sam.png', dpi=300)
plt.show()


# Plot Figure 4 (c)

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Load datasets
latent = xr.open_dataset('/mnt/d/Data/latent_heat_2002_2023.nc').sel(latitude=slice(-62, -67)).mslhf
sensible = xr.open_dataset('/mnt/d/Data/sensible_heat_2002_2023.nc').sel(latitude=slice(-62, -67)).msshf
shortwave = xr.open_dataset('/mnt/d/Data/shortwave_heat_2002_2023.nc').sel(latitude=slice(-62, -67)).msnswrf
longwave = xr.open_dataset('/mnt/d/Data/longwave_heat_2002_2023.nc').sel(latitude=slice(-62, -67)).msnlwrf

# Ensure time alignment by reindexing time (if necessary)
time_index = latent.valid_time
sensible = sensible.reindex(valid_time=time_index, method="nearest")
shortwave = shortwave.reindex(valid_time=time_index, method="nearest")
longwave = longwave.reindex(valid_time=time_index, method="nearest")

# Calculate the spatial mean for each heat flux component
latent_mean = latent.mean(dim=['latitude', 'longitude'])
sensible_mean = sensible.mean(dim=['latitude', 'longitude'])
shortwave_mean = shortwave.mean(dim=['latitude', 'longitude'])
longwave_mean = longwave.mean(dim=['latitude', 'longitude'])

# Calculate net heat flux: positive values indicate heat input to the ocean
net_heat_flux = shortwave_mean + longwave_mean + sensible_mean + latent_mean

# Calculate monthly climatology (2002–2022) for net heat flux
net_climatology = net_heat_flux.sel(valid_time=slice('2002', '2022')).groupby('valid_time.month').mean()

# Calculate anomalies for the entire dataset
net_anomalies = net_heat_flux.groupby('valid_time.month') - net_climatology

# Apply a 90-day moving average to the net heat flux anomalies
net_anomalies_ma = net_anomalies.rolling(valid_time=90, min_periods=1, center=False).mean()

# Plotting the net heat flux anomaly for the entire dataset
plt.figure(figsize=(18, 6))

# Plot net heat flux anomalies
plt.plot(
    net_anomalies_ma.valid_time, net_anomalies_ma, 
    label='Net Heat Flux Anomalies (90-day MA)', color='red', linewidth=2
)

# Add zero line for reference
plt.axhline(0, color='black', linestyle='--', linewidth=1)

# Customize the plot
plt.ylabel('Net Heat Flux Anomaly (W/m²)', fontsize=15)
plt.grid(True, linestyle='--', alpha=0.6)

# Set x-axis limits to remove empty spaces
plt.xlim(net_anomalies_ma.valid_time[0].values, net_anomalies_ma.valid_time[-1].values)

# Format x-ticks to show years
years = pd.date_range(start="2002-01-01", end="2023-12-31", freq="YS")
plt.xticks(years, [date.strftime("%Y") for date in years], rotation=45, fontsize=14)  # Increased font size
plt.yticks(fontsize=14)  # Increased font size for y-axis ticks

# Add legend and layout adjustments
# plt.legend(fontsize=12)
plt.tight_layout()

# Save the plot
plt.savefig('/home/soumya/Backup/Plots/Net_Heat_Flux_Anomalies_2002_2023_90dMA.png', dpi=300)
plt.show()


# Plotting Figure S3

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Load wind stress data
u = xr.open_mfdataset('/mnt/d/Data/u10.nc').sel(valid_time=slice('2016-01-01', '2016-12-31'), longitude=slice(30, 60), latitude=slice(-60, -67)).u10
v = xr.open_mfdataset('/mnt/d/Data/v10.nc').sel(valid_time=slice('2016-01-01', '2016-12-31'), longitude=slice(30, 60), latitude=slice(-60, -67)).v10

# Calculate wind stress components
wd = (u**2 + v**2) ** 0.5
cd = 1.3 * (10**-3)
tau_x = 1.3 * cd * u * wd
tau_y = 1.3 * cd * v * wd

# Calculate grid spacing
dlat = u.latitude.diff('latitude')
dlon = u.longitude.diff('longitude')
dy_m = np.abs(111320 * dlat)
dx_m = 111320 * dlon * np.cos(np.deg2rad(u.latitude))

# Calculate wind stress curl
d_tau_y_dx = tau_y.differentiate('longitude') / dx_m
d_tau_x_dy = tau_x.differentiate('latitude') / dy_m
wind_stress_curl = d_tau_y_dx - d_tau_x_dy
curl = wind_stress_curl.mean(dim=('longitude', 'latitude'))

# Filter curl for JJAS
curl_jjas = curl.sel(valid_time=curl['valid_time'].dt.month.isin([6, 7, 8, 9]))

# Load polynya area data
polynya_df = pd.read_csv('/mnt/d/Data/Polynya Area/AMSR/Polynya_Area_30.csv')
polynya_df['Timestamp'] = pd.to_datetime(polynya_df['Timestamp'])

# Filter data for 2023 and JJAS months
polynya_df = polynya_df[(polynya_df['Timestamp'].dt.year == 2016) & 
                        (polynya_df['Timestamp'].dt.month.isin([6, 7, 8, 9]))]
polynya_df = polynya_df[(polynya_df['lat'] >= -67) & (polynya_df['lat'] <= -60) & 
                        (polynya_df['lon'] >= 30) & (polynya_df['lon'] <= 60)]
polynya_area = polynya_df.groupby('Timestamp')['total_area'].sum().reset_index()
polynya_area['total_area'] = polynya_area['total_area'] / (10**9)  # Convert to square kilometers

# Plotting
fig, ax1 = plt.subplots(figsize=(20, 10))

# Plot wind stress curl on primary y-axis
ax1.plot(curl_jjas['valid_time'], curl_jjas * 10**5, color='tab:blue', linewidth=2, linestyle='-', label='Wind Stress Curl (10$^{-5}$ N m$^{-3}$)')
# ax1.invert_yaxis()

# Plot polynya area on secondary y-axis
ax2 = ax1.twinx()
ax2.plot(polynya_area['Timestamp'], polynya_area['total_area'], color='tab:orange', linewidth=2, linestyle='-', label='Polynya Area (x $10^{3}$ km²)')

# Format the x-axis for full dates
ax1.xaxis.set_major_locator(mdates.MonthLocator())
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
# fig.autofmt_xdate()  # Rotate date labels

# Add grid, labels, and title
ax1.grid(True, linestyle='--', alpha=0.7)
# ax1.set_title('Wind Stress Curl and Polynya Area (JJAS 2023)', fontsize=24, weight='bold')  # Increased font size
# ax1.set_xlabel('Time (Months)', fontsize=20)  # Increased font size
ax1.set_ylabel('Wind Stress Curl (10$^{-5}$ N/m$^{3}$)', fontsize=16, color='tab:blue')  # Increased font size
ax2.set_ylabel('Polynya Area (x $10^{3}$ km²)', fontsize=16, color='tab:orange')  # Increased font size

# Enhance readability
ax1.tick_params(axis='both', which='major', labelsize=16)  # Larger tick labels
ax2.tick_params(axis='y', labelsize=16)  # Larger tick labels

plt.tight_layout()
# Save and show the plot
plt.savefig('/home/soumya/Backup/Plots/Cosmonaut Polynya/wind_curl_polynya_area_JJAS_2016_large_font.png', dpi=400)
plt.show()


# Plotting Figure S4

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Load wind data
u_path = '/mnt/d/Data/u10.nc'  # Update this to your U-wind dataset
v_path = '/mnt/d/v10.nc'  # Update this to your V-wind dataset
u_ds = xr.open_dataset(u_path)
v_ds = xr.open_dataset(v_path)

# Define the spatial and temporal range for 2016
latitude_range = slice(-60, -67)  # Antarctic region
longitude_range = slice(30, 60)  # Region of interest
time_range = slice('2016-01-01', '2016-12-31')  # Time range for 2016

# Subset data for 2016
u = u_ds.sel(valid_time=time_range, latitude=latitude_range, longitude=longitude_range).u10
v = v_ds.sel(valid_time=time_range, latitude=latitude_range, longitude=longitude_range).v10

# Constants
rho_water = 1025  # Density of seawater in kg/m^3
omega = 7.292e-5  # Earth's angular velocity in rad/s

# Calculate Coriolis parameter
latitude_radians = np.deg2rad(u.latitude)
f = 2 * omega * np.sin(latitude_radians)

# Calculate wind stress components
wd = (u**2 + v**2) ** 0.5
cd = 1.3 * (10**-3)
tau_x = cd * u * wd
tau_y = cd * v * wd

# Divide wind stress components by f and rho_water
adjusted_tau_x = tau_x / (f * rho_water)
adjusted_tau_y = tau_y / (f * rho_water)

# Calculate grid spacing
dlat = adjusted_tau_x.latitude.diff('latitude')
dlon = adjusted_tau_x.longitude.diff('longitude')
dy_m = np.abs(111320 * dlat)  # Approximate conversion from degrees to meters
dx_m = 111320 * dlon * np.cos(np.deg2rad(adjusted_tau_x.latitude))

# Calculate curl of the adjusted wind stress
d_adj_tau_y_dx = adjusted_tau_y.differentiate('longitude') / dx_m
d_adj_tau_x_dy = adjusted_tau_x.differentiate('latitude') / dy_m
ekman_pumping_curl = (d_adj_tau_y_dx - d_adj_tau_x_dy) * 86400

# Calculate the daily mean of Ekman pumping curl over the region
daily_mean_curl = ekman_pumping_curl.mean(dim=['latitude', 'longitude'])

# Plot the daily mean Ekman pumping curl for 2016
plt.figure(figsize=(14, 7))
plt.plot(
    daily_mean_curl.valid_time, daily_mean_curl,
    label='Daily Mean Ekman Pumping Curl (2016)', color='darkblue', linewidth=1.5
)

# Customize the plot 
plt.ylabel('Ekman Upwelling Velocity (m/day)', fontsize=14)
plt.xticks(fontsize=14)  
plt.yticks(fontsize=14) 
plt.grid(True, linestyle='--', alpha=0.6)

# Ensure x-axis fits the data range
plt.xlim(daily_mean_curl.valid_time.min().values, daily_mean_curl.valid_time.max().values)

# Save the plot
plt.tight_layout()
plt.savefig('/home/soumya/Backup/Plots/Ekman_Pumping_Curl_2016_Daily_Tight_LargerFonts.png', dpi=300)
plt.show()


# Plotting Figure S5

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Load datasets
latent = xr.open_dataset('/mnt/d/Data/latent_heat_2002_2023.nc').sel(latitude=slice(-62, -67)).mslhf
sensible = xr.open_dataset('/mnt/d/Data/sensible_heat_2002_2023.nc').sel(latitude=slice(-62, -67)).msshf
shortwave = xr.open_dataset('/mnt/d/Data/shortwave_heat_2002_2023.nc').sel(latitude=slice(-62, -67)).msnswrf
longwave = xr.open_dataset('/mnt/d/Data/longwave_heat_2002_2023.nc').sel(latitude=slice(-62, -67)).msnlwrf

# Ensure time alignment by reindexing time (if necessary)
time_index = latent.valid_time
sensible = sensible.reindex(valid_time=time_index, method="nearest")
shortwave = shortwave.reindex(valid_time=time_index, method="nearest")
longwave = longwave.reindex(valid_time=time_index, method="nearest")

# Calculate the spatial mean for each heat flux component
latent_mean = latent.mean(dim=['latitude', 'longitude'])
sensible_mean = sensible.mean(dim=['latitude', 'longitude'])
shortwave_mean = shortwave.mean(dim=['latitude', 'longitude'])
longwave_mean = longwave.mean(dim=['latitude', 'longitude'])

# Calculate climatologies for each component (2002–2022)
latent_climatology = latent_mean.sel(valid_time=slice('2002', '2022')).groupby('valid_time.month').mean()
sensible_climatology = sensible_mean.sel(valid_time=slice('2002', '2022')).groupby('valid_time.month').mean()
shortwave_climatology = shortwave_mean.sel(valid_time=slice('2002', '2022')).groupby('valid_time.month').mean()
longwave_climatology = longwave_mean.sel(valid_time=slice('2002', '2022')).groupby('valid_time.month').mean()

# Calculate anomalies for each component
latent_anomalies = latent_mean.groupby('valid_time.month') - latent_climatology
sensible_anomalies = sensible_mean.groupby('valid_time.month') - sensible_climatology
shortwave_anomalies = shortwave_mean.groupby('valid_time.month') - shortwave_climatology
longwave_anomalies = longwave_mean.groupby('valid_time.month') - longwave_climatology

# Apply a 90-day moving average to each component
latent_anomalies_ma = latent_anomalies.rolling(valid_time=90, min_periods=1, center=False).mean()
sensible_anomalies_ma = sensible_anomalies.rolling(valid_time=90, min_periods=1, center=False).mean()
shortwave_anomalies_ma = shortwave_anomalies.rolling(valid_time=90, min_periods=1, center=False).mean()
longwave_anomalies_ma = longwave_anomalies.rolling(valid_time=90, min_periods=1, center=False).mean()

# Plot each anomaly in a separate subplot
fig, axes = plt.subplots(4, 1, figsize=(16, 14), sharex=True)

# Plot latent heat flux anomalies
axes[0].plot(latent_anomalies_ma.valid_time, latent_anomalies_ma, color='blue')
axes[0].axhline(0, color='black', linestyle='--', linewidth=1)
axes[0].set_ylabel('Latent Heat Flux (W/m²)', fontsize=14)
axes[0].grid(True, linestyle='--', alpha=0.6)

# Plot sensible heat flux anomalies
axes[1].plot(sensible_anomalies_ma.valid_time, sensible_anomalies_ma, color='orange')
axes[1].axhline(0, color='black', linestyle='--', linewidth=1)
axes[1].set_ylabel('Sensible Heat Flux (W/m²)', fontsize=14)
axes[1].grid(True, linestyle='--', alpha=0.6)

# Plot shortwave heat flux anomalies
axes[2].plot(shortwave_anomalies_ma.valid_time, shortwave_anomalies_ma, color='green')
axes[2].axhline(0, color='black', linestyle='--', linewidth=1)
axes[2].set_ylabel('Shortwave Heat Flux (W/m²)', fontsize=14)
axes[2].grid(True, linestyle='--', alpha=0.6)

# Plot longwave heat flux anomalies
axes[3].plot(longwave_anomalies_ma.valid_time, longwave_anomalies_ma, color='red')
axes[3].axhline(0, color='black', linestyle='--', linewidth=1)
axes[3].set_ylabel('Longwave Heat Flux (W/m²)', fontsize=14)
axes[3].grid(True, linestyle='--', alpha=0.6)

# Customize x-axis for the bottom subplot
axes[3].set_xlabel('Time', fontsize=16)

# Add x-ticks for all years
years = pd.date_range(start="2002-01-01", end="2023-12-31", freq="YS")
axes[3].set_xticks(years)
axes[3].set_xticklabels([date.strftime("%Y") for date in years], rotation=45, fontsize=14)

# Set x-axis limits to remove extra spaces
start_time = pd.Timestamp("2002-01-01")
end_time = pd.Timestamp("2023-12-31")
for ax in axes:
    ax.set_xlim([start_time, end_time])
    ax.tick_params(axis='y', labelsize=12)


# Adjust layout and save the plot
plt.tight_layout()
plt.savefig('/home/soumya/Backup/Plots/Heat_Flux_Anomalies_Components_2002_2023_90dMA.png', dpi=300, bbox_inches='tight')
plt.show()
